# IMDB Movies

Goal: Create a MySQL database that catalogues movies, and analyze what makes a movie successful or not.

## Importing Libraries

In [1]:
#run this if you don't already have tmdbsimple on your computer
#!pip instal tmdbsimple

In [32]:
import numpy as np
import pandas as pd

import os, time,json
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

In [3]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['title_basics.csv.gz', 'title_akas.csv.gz', 'title_ratings.csv.gz']

In [4]:
#Setting pandas options to max column and row displays
pd.set_option('display.max_columns', None) #Used for displaying columns
pd.set_option('display.max_rows', None) #Used for displaying rows

## Creating and Cleaning Pandas Databases

* replace all "\N" values with np.nan
* remove all movies that are not localized in the US
* For Basics:
    * remove movies without a genre
    * only keep movies
    * only use movies from 2000-2022 (inclusive)
    

In [5]:
# Creating url paths from original links
# DEFUNCT: data has been cleaned and is available in data folder
#basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
#ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"
#akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"

In [6]:
#Reading url paths into databases
# DEFUNCT: data has been cleaned and is available in data folder
#basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
#ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)
#akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)

In [7]:
#Determining where to search for US titles
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [8]:
#Checking what abbreviation = US
#i.e. is the abbr. US, or USA?
akas["region"].value_counts()

US    1424064
Name: region, dtype: int64

In [9]:
#Cleaning akas
is_in_US = akas["region"] == "US"
akas = akas[is_in_US]
akas = akas.replace({"\\N":np.nan})

akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [10]:
#Cleaning ratings
US_filter = ratings["tconst"].isin(akas["titleId"])
ratings = ratings[US_filter]

ratings = ratings.replace({"\\N": np.nan})

ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1961
1,tt0000002,5.8,263
2,tt0000005,6.2,2600
3,tt0000006,5.1,178
4,tt0000007,5.4,817


In [11]:
#Cleaning basics
US_filter = basics["tconst"].isin(akas["titleId"])
basics = basics[US_filter]
basics = basics.replace({"\\N": np.nan})

basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [12]:
#Filtering for only movies
isMovie = basics["titleType"] == "movie"
basics = basics[isMovie]

basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [13]:
#Removing documentaries
isDocumentary = basics["genres"].str.contains("documentary", na = False)
basics = basics[~isDocumentary]

basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [14]:
#Changing startYear to be an int and from range 2000-2022
basics["startYear"] = basics["startYear"].astype(float)
#Removing movies before 2000
isOlderThan2000 = basics["startYear"] >= 2000
basics = basics[isOlderThan2000]

#Removing movies after 2022
isYoungerThan2022 = basics["startYear"] <= 2021
basics = basics[isYoungerThan2022]

basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113650 entries, 0 to 113649
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          113650 non-null  object 
 1   titleType       113650 non-null  object 
 2   primaryTitle    113650 non-null  object 
 3   originalTitle   113650 non-null  object 
 4   isAdult         113650 non-null  int64  
 5   startYear       113650 non-null  float64
 6   endYear         0 non-null       float64
 7   runtimeMinutes  113650 non-null  int64  
 8   genres          113650 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 8.7+ MB


In [15]:
#removing null runtimeMinutes
isNullMinutes = basics["runtimeMinutes"] > 0
basics = basics[isNullMinutes]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [16]:
isNullGenre = basics["genres"] > ""
basics = basics[isNullGenre]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [17]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [18]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113650 entries, 0 to 113649
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          113650 non-null  object 
 1   titleType       113650 non-null  object 
 2   primaryTitle    113650 non-null  object 
 3   originalTitle   113650 non-null  object 
 4   isAdult         113650 non-null  int64  
 5   startYear       113650 non-null  float64
 6   endYear         0 non-null       float64
 7   runtimeMinutes  113650 non-null  int64  
 8   genres          113650 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 8.7+ MB


In [19]:
#No need to do this again

#Committing files to data folder
#basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
#akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)
#ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [20]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113650 entries, 0 to 113649
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          113650 non-null  object 
 1   titleType       113650 non-null  object 
 2   primaryTitle    113650 non-null  object 
 3   originalTitle   113650 non-null  object 
 4   isAdult         113650 non-null  int64  
 5   startYear       113650 non-null  float64
 6   endYear         0 non-null       float64
 7   runtimeMinutes  113650 non-null  int64  
 8   genres          113650 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 8.7+ MB


In [21]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1424064 entries, 0 to 1424063
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   titleId          1424064 non-null  object 
 1   ordering         1424064 non-null  int64  
 2   title            1424064 non-null  object 
 3   region           1424064 non-null  object 
 4   language         3866 non-null     object 
 5   types            976227 non-null   object 
 6   attributes       46241 non-null    object 
 7   isOriginalTitle  1422719 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 97.8+ MB


In [22]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 492984 entries, 0 to 492983
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         492984 non-null  object 
 1   averageRating  492984 non-null  float64
 2   numVotes       492984 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.0+ MB


## Extracting data from TMDB

### Data Extraction


In [23]:
def get_movie_with_rating(movie_id):
    """adapted from github.com/celiao/tmdbsimple"""
    #Get the movie object associated with movie_id
    movie = tmdb.Movies(movie_id)
    
    #save the .info and .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    
    #Loop through the countries in releases
    for country in releases["countries"]:
        if country["iso_3166_1"] == "US":
            info["certification"] = country["certification"]
    return info

In [24]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [25]:
with open('/Users/joseph/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [26]:
tmdb.API_KEY =  login['api-key']

In [34]:
test_ids = ["tt0848228", "tt0332280"]
results = []
errors = []
for movie_id in test_ids:
    
    try:
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)
        
    except Exception as e: 
        errors.append([movie_id, e])
    
pd.DataFrame(results)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,When an unexpected enemy emerges and threatens...,151.156,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,"[{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZ...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-04-25,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.707,28376,PG-13
1,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,An epic love story centered around an older ma...,57.461,/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg,"[{'id': 12, 'logo_path': '/5ThIuO93vsk47oexKTS...","[{'iso_3166_1': 'US', 'name': 'United States o...",2004-06-25,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.877,10222,PG-13


In [28]:
YEARS_TO_GET = [2000,2001]
errors = [ ]


In [33]:
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
        # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()

    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)

    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])

    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1766 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1902 [00:00<?, ?it/s]

In [35]:
print(f"- Total errors: {len(errors)}")

- Total errors: 0
